In [87]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
df = pd.read_csv('t_car_last.csv')
d_plt = df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46781 entries, 0 to 46780
Data columns (total 10 columns):
id               46781 non-null int64
city             46781 non-null object
brand            46781 non-null object
output_volume    46781 non-null float64
launch_year      46781 non-null int64
kilometres       46781 non-null float64
is_import        46781 non-null int64
old_price        46781 non-null float64
gear_type        46781 non-null int64
price            46781 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 3.6+ MB


In [88]:
df = df.drop(['id'], axis=1)
le = preprocessing.LabelEncoder()
le_count = 0

label_code_map = {}  # 保存编码的数据，在预测时需要将数据根据训练数据编码
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) >= 10:
            # Train on the training data
            le.fit(df[col])
            # Transform both training and testing data
            orl = df[col]
            df[col] = le.transform(orl)
            label_code_map[col] = le.classes_
            # Keep track of how many columns were label encoded
            le_count += 1
df

,city,brand,output_volume,launch_year,kilometres,is_import,old_price,gear_type,price
0,0,119,1.6,2015,2.2,0,14.42,1,8.29
1,0,230,2.0,2015,4.4,0,24.18,0,13.39
2,0,230,1.5,2016,3.0,0,20.60,0,12.54
3,0,113,1.5,2017,3.2,0,11.83,1,6.99
4,0,113,1.5,2016,4.0,0,11.92,1,5.83
...,...,...,...,...,...,...,...,...,...
46776,30,210,1.6,2014,2.7,1,30.49,0,10.80
46777,30,93,1.4,2018,2.1,0,11.50,0,7.50
46778,30,32,1.5,2017,4.8,0,6.72,0,3.58
46779,30,222,2.2,2013,5.0,0,10.83,1,4.10


In [89]:
# 数据归一化处理:(原值-平均值)/数据标准差 Z-score标准化方法
def z_scrore_normalize_feature(df):
    # return df.apply(lambda column: (column - column.mean())/column.std())
    ss = preprocessing.StandardScaler().fit(df)
    return ss.transform(df), ss.mean_, ss.var_
dz, mean_, var_ = z_scrore_normalize_feature(df.iloc[:, :8])
dz

array([[-1.67854063, -0.53230479, -0.4136155 , ..., -0.19537267,
        -0.29470644,  1.87184477],
       [-1.67854063,  0.81671959,  0.74143146, ..., -0.19537267,
         0.64754045, -0.53423233],
       [-1.67854063,  0.81671959, -0.70237724, ..., -0.19537267,
         0.3019212 , -0.53423233],
       ...,
       [ 1.66952295, -1.58964823, -0.70237724, ..., -0.19537267,
        -1.03807745, -0.53423233],
       [ 1.66952295,  0.71949261,  1.31895493, ..., -0.19537267,
        -0.6412911 ,  1.87184477],
       [ 1.66952295,  1.76468267,  0.74143146, ..., -0.19537267,
         0.2391691 , -0.53423233]])

In [90]:
# max和min的归一化操作
def min_max_normalize_feature(df):
    df = (df - df.min())/(df.max() - df.min())
    return df
dmm = min_max_normalize_feature(df.iloc[:, :8])
dmm

,city,brand,output_volume,launch_year,kilometres,is_import,old_price,gear_type
0,0.0,0.385113,0.132075,0.714286,0.110106,0.0,0.058758,1.0
1,0.0,0.744337,0.207547,0.714286,0.220714,0.0,0.110516,0.0
2,0.0,0.744337,0.113208,0.785714,0.150327,0.0,0.091531,0.0
3,0.0,0.365696,0.113208,0.857143,0.160382,0.0,0.045023,1.0
4,0.0,0.365696,0.113208,0.785714,0.200603,0.0,0.045500,1.0
...,...,...,...,...,...,...,...,...
46776,1.0,0.679612,0.132075,0.642857,0.135244,1.0,0.143978,0.0
46777,1.0,0.300971,0.094340,0.928571,0.105078,0.0,0.043273,0.0
46778,1.0,0.103560,0.113208,0.857143,0.240825,0.0,0.017924,0.0
46779,1.0,0.718447,0.245283,0.571429,0.250880,0.0,0.039720,1.0


In [91]:
df.describe()

,city,brand,output_volume,launch_year,kilometres,is_import,old_price,gear_type,price
count,46781.000000,46781.000000,46781.000000,46781.000000,46781.000000,46781.000000,46781.000000,46781.000000,46781.000000
mean,15.040401,162.798935,1.743238,2014.606892,5.753192,0.036767,17.472634,0.222035,7.962740
std,8.960499,82.282565,0.346310,2.113985,3.142012,0.188191,10.358330,0.415619,4.785922
min,0.000000,0.000000,0.900000,2005.000000,0.010000,0.000000,3.340000,0.000000,0.700000
25%,7.000000,119.000000,1.500000,2013.000000,3.400000,0.000000,11.100000,0.000000,4.880000
50%,15.000000,170.000000,1.600000,2015.000000,5.300000,0.000000,14.850000,0.000000,6.740000
75%,23.000000,230.000000,2.000000,2016.000000,7.600000,0.000000,20.930000,0.000000,9.680000
max,30.000000,309.000000,6.200000,2019.000000,19.900000,1.000000,191.910000,1.000000,140.000000


In [92]:
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1) # 提取label
X_data = np.array(dz) # 提取feature
X_data

array([[-1.67854063, -0.53230479, -0.4136155 , ..., -0.19537267,
        -0.29470644,  1.87184477],
       [-1.67854063,  0.81671959,  0.74143146, ..., -0.19537267,
         0.64754045, -0.53423233],
       [-1.67854063,  0.81671959, -0.70237724, ..., -0.19537267,
         0.3019212 , -0.53423233],
       ...,
       [ 1.66952295, -1.58964823, -0.70237724, ..., -0.19537267,
        -1.03807745, -0.53423233],
       [ 1.66952295,  0.71949261,  1.31895493, ..., -0.19537267,
        -0.6412911 ,  1.87184477],
       [ 1.66952295,  1.76468267,  0.74143146, ..., -0.19537267,
         0.2391691 , -0.53423233]])

In [93]:
# 将数据集分成训练集和测试集 0.8
def shuffle_data(features: np.array, labels: np.array, test_size=0.8):
    np.random.seed(0)
    index = list(range(len(features)))
    np.random.shuffle(index)

    data_size = len(features)
    train_data_size = int(test_size * data_size)

    train_features = X_data[:train_data_size, :]
    train_labels = y_data[:train_data_size, :]

    test_features = X_data[train_data_size:, :]
    test_labels = y_data[train_data_size:, :]

    return train_features, train_labels, test_features, test_labels

In [94]:
train_features, train_labels, test_features, test_labels = shuffle_data(X_data, y_data)

In [95]:
def transfer_torch_data(train_features, train_labels, test_features, test_labels):
    return torch.from_numpy(train_features).float(),torch.from_numpy(train_labels).float(), torch.from_numpy(test_features).float(),torch.from_numpy(test_labels).float()

In [96]:
train_features, train_labels, test_features, test_labels = transfer_torch_data(train_features, train_labels, test_features, test_labels)

In [97]:
model = nn.Sequential(
    nn.Linear(8, 128),
    nn.Sigmoid(),
    nn.Linear(128, 1)
)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [98]:
epochs = 2000
for each_epoch in range(epochs):
    optimizer.zero_grad()
    predictions = model(train_features)
    loss_value = loss(predictions, train_labels)
    loss_value.backward()
    optimizer.step()
    if each_epoch % 100 == 0:
        print(f"epoch: {each_epoch}, loss: {loss_value.data}")

model.eval()

epoch: 0, loss: 86.07601165771484
epoch: 100, loss: 6.63686466217041
epoch: 200, loss: 4.373867034912109
epoch: 300, loss: 3.8928585052490234
epoch: 400, loss: 3.5706560611724854
epoch: 500, loss: 3.3147311210632324
epoch: 600, loss: 3.10091233253479
epoch: 700, loss: 2.911760091781616
epoch: 800, loss: 2.7588274478912354
epoch: 900, loss: 2.637605667114258
epoch: 1000, loss: 2.5398027896881104
epoch: 1100, loss: 2.45219349861145
epoch: 1200, loss: 2.3688852787017822
epoch: 1300, loss: 2.2851548194885254
epoch: 1400, loss: 2.209597110748291
epoch: 1500, loss: 2.140619993209839
epoch: 1600, loss: 2.07092547416687
epoch: 1700, loss: 2.003049850463867
epoch: 1800, loss: 1.9390051364898682
epoch: 1900, loss: 1.8783907890319824


Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=128, out_features=1, bias=True)
)

In [99]:
y_pred = model(test_features).detach().numpy()
y_pred

array([[10.039479 ],
       [ 4.7735047],
       [18.555267 ],
       ...,
       [ 4.7878137],
       [ 3.0858583],
       [ 8.468801 ]], dtype=float32)

In [100]:
test_labels
# plt.plot(train_features, train_labels, 'ro', label='Original Data')
# plt.plot(test_features, y_pred, 'b-', label='Fitted Line')
# plt.legend()
# plt.show()

tensor([[ 9.7000],
        [ 5.7800],
        [18.5000],
        ...,
        [ 3.5800],
        [ 4.1000],
        [11.8000]])

In [107]:
# 输入数据，预测
a = np.array([0, 230, 2.0, 2015, 4.4, 0, 24.18, 0])
b = (a - mean_)/np.sqrt(var_)
b = torch.from_numpy(b).float()
y_b = model(b).detach().numpy()
y_b

array([11.372401], dtype=float32)